In [59]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.ensemble import RandomForestClassifier

In [60]:
data = pd.read_csv("heart_attack_prediction_india.csv")
data.head()

,Patient_ID,State_Name,Age,Gender,Diabetes,Hypertension,Obesity,Smoking,Alcohol_Consumption,Physical_Activity,...,Diastolic_BP,Air_Pollution_Exposure,Family_History,Stress_Level,Healthcare_Access,Heart_Attack_History,Emergency_Response_Time,Annual_Income,Health_Insurance,Heart_Attack_Risk
0,1,Rajasthan,42,Female,0,0,1,1,0,0,...,119,1,0,4,0,0,157,611025,0,0
1,2,Himachal Pradesh,26,Male,0,0,0,0,1,1,...,115,0,0,7,0,0,331,174527,0,0
2,3,Assam,78,Male,0,0,1,0,0,1,...,117,0,1,10,1,0,186,1760112,1,0
3,4,Odisha,58,Male,1,0,1,0,0,1,...,65,0,0,1,1,1,324,1398213,0,0
4,5,Karnataka,22,Male,0,0,0,0,0,1,...,109,0,0,9,0,0,209,97987,0,1


In [61]:
data.tail()

,Patient_ID,State_Name,Age,Gender,Diabetes,Hypertension,Obesity,Smoking,Alcohol_Consumption,Physical_Activity,...,Diastolic_BP,Air_Pollution_Exposure,Family_History,Stress_Level,Healthcare_Access,Heart_Attack_History,Emergency_Response_Time,Annual_Income,Health_Insurance,Heart_Attack_Risk
9995,9996,Maharashtra,20,Female,0,0,0,1,0,1,...,74,1,1,10,0,0,167,1175023,1,0
9996,9997,Rajasthan,34,Female,0,0,0,0,0,0,...,78,0,1,1,0,0,123,999441,0,1
9997,9998,Gujarat,76,Male,0,0,0,0,0,0,...,98,1,0,1,0,1,376,159536,0,0
9998,9999,Andhra Pradesh,52,Male,0,0,0,0,0,1,...,60,1,0,7,1,0,313,153207,0,0
9999,10000,Haryana,25,Female,0,1,0,1,0,1,...,89,1,0,4,0,1,366,1732794,1,0


In [62]:
data = data.drop(['Annual_Income','Health_Insurance'],axis=1)
print(data.columns)


Index(['Patient_ID', 'State_Name', 'Age', 'Gender', 'Diabetes', 'Hypertension',
       'Obesity', 'Smoking', 'Alcohol_Consumption', 'Physical_Activity',
       'Diet_Score', 'Cholesterol_Level', 'Triglyceride_Level', 'LDL_Level',
       'HDL_Level', 'Systolic_BP', 'Diastolic_BP', 'Air_Pollution_Exposure',
       'Family_History', 'Stress_Level', 'Healthcare_Access',
       'Heart_Attack_History', 'Emergency_Response_Time', 'Heart_Attack_Risk'],
      dtype='object')


In [63]:
#data cleaning
print(data.isnull().sum())

Patient_ID                 0
State_Name                 0
Age                        0
Gender                     0
Diabetes                   0
Hypertension               0
Obesity                    0
Smoking                    0
Alcohol_Consumption        0
Physical_Activity          0
Diet_Score                 0
Cholesterol_Level          0
Triglyceride_Level         0
LDL_Level                  0
HDL_Level                  0
Systolic_BP                0
Diastolic_BP               0
Air_Pollution_Exposure     0
Family_History             0
Stress_Level               0
Healthcare_Access          0
Heart_Attack_History       0
Emergency_Response_Time    0
Heart_Attack_Risk          0
dtype: int64


In [64]:
X = data.drop('Heart_Attack_Risk',axis=1)
y =  data['Heart_Attack_Risk']

In [65]:
obj_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(exclude=['object']).columns

In [66]:
print(obj_cols,num_cols)

Index(['State_Name', 'Gender'], dtype='object') Index(['Patient_ID', 'Age', 'Diabetes', 'Hypertension', 'Obesity', 'Smoking',
       'Alcohol_Consumption', 'Physical_Activity', 'Diet_Score',
       'Cholesterol_Level', 'Triglyceride_Level', 'LDL_Level', 'HDL_Level',
       'Systolic_BP', 'Diastolic_BP', 'Air_Pollution_Exposure',
       'Family_History', 'Stress_Level', 'Healthcare_Access',
       'Heart_Attack_History', 'Emergency_Response_Time'],
      dtype='object')


In [67]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

num_cols_imputer = SimpleImputer(strategy='mean')
cat_cols_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_cols_imputer, num_cols),
    ('cat', cat_cols_pipeline, obj_cols)
])

In [69]:
X_transformed = preprocessor.fit_transform(X)
y = y.astype(int)

In [70]:
#splitting the dataset 
from sklearn.model_selection import train_test_split,GridSearchCV

X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.30,random_state=42)

In [57]:
print(X_train.shape,y_train.shape)

(7000, 23) (7000,)


In [ ]:
# n_estimators = [500,750,1000]
# criterion= ['gini','entropy','log_loss']
# max_depth = [20,40,60]
# max_features = ['sqrt','log2',None]
# bootstrap = [True]

In [ ]:
# param_grid = {
#     'model__n_estimators': [500, 750, 1000],
#     'model__criterion': ['gini', 'entropy', 'log_loss'],
#     'model__max_depth': [20, 40, 60],
#     'model__max_features': ['sqrt', 'log2', None],
#     'model__bootstrap': [True]
# }

In [ ]:
# my_model = Pipeline(steps=[('preprocessor',preprocessor),('model',RandomForestClassifier())])

In [ ]:

# grid_search = GridSearchCV(estimator=my_model,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)

In [48]:
# grid_search.fit(X_train,y_train)

In [ ]:
# grid_search.best_params_

{'model__bootstrap': True,
 'model__criterion': 'log_loss',
 'model__max_depth': 20,
 'model__max_features': None,
 'model__n_estimators': 1000}

In [ ]:
grid_search.best_score_

np.float64(0.6998571428571428)

In [72]:
best_model = RandomForestClassifier(class_weight='balanced',bootstrap=True,criterion='log_loss',max_depth=20,max_features=None,n_estimators=1000, random_state=42)
best_model.fit(X_train, y_train)

ValueError: could not convert string to float: 'Sikkim'

In [ ]:
from sklearn.metrics import f1_score,precision_score, accuracy_score, confusion_matrix, recall_score

preds = best_model.predict(X_valid)
print("Accuracy Score:",accuracy_score(y_valid,preds))
print("confusion_matrix:",confusion_matrix(y_valid,preds))
print("F1 score:",f1_score(y_valid,preds))
print("precision score:",precision_score(y_valid,preds))
print("recall score:",recall_score(y_valid,preds))

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.